In [1]:
import neo4j_connector as nc
import re
import docx
import os
from nltk.corpus import stopwords
from nltk.tag import StanfordPOSTagger
from nltk.tokenize import word_tokenize
import spacy
import pickle
from IPython.display import display
from tqdm import tqdm

In [2]:
def extract_dialogs(doc):
    inicio = False
    dialog = ''
    dialogs = []
    for par in doc.paragraphs:   
        if par.text == '###':
            inicio = True
        else:
            inicio = False

        if inicio:
            if dialog == '':
                pass
            else:
                dialogs.append(dialog)
            dialog = ''
        else:
            dialog = dialog + par.text + ' '
    return dialogs

In [3]:
def separate_dialogs(dialogs):
    output = []
    for dialog in dialogs:
        pos_fin = dialog.find(':')
        output.append([dialog[:pos_fin].lower(), dialog[pos_fin + 1 :].lower()])
    return output

In [4]:
def clean_parenthesis(text):
    regex = r" \(([^\)]+)\)"
    matches = re.finditer(regex, text)
    output = text
    for matchNum, match in enumerate(matches, start=1):
        t_parentesis = match.group()
        output = output.replace(t_parentesis, '')
    return output

In [5]:
#aunque no te lo creas a veces no se escribe del todo correctamente
def clean_mr_mrs(text):
    return text.replace('el señor ', '').replace('la señora ', '') \
.replace('el señora', '').replace('la señor', '') \
.replace('señor ', '').replace('señora ', '')

In [6]:
def clean_text(text):
    return text.replace('?', '').replace('¿', '').replace('«', '').replace('»', '') \
.replace('.', ' ').replace(',', ' ').replace('—', ' ').replace('-', ' ').replace('%', ' ') \
.replace('š', ' ').replace(';', ' ').replace('!', ' ').replace('¡', ' ') \
.replace('–', ' ').replace(' : ', ': ').replace(':', ' ').replace('…', ' ').replace('  ', ' ')

In [7]:
def clean_names(name):
    return name.replace('presidente del gobierno en funciones', 'sánchez pérez-castejón') \
        .replace('ministra de justicia en funciones', 'delgado garcía') \
        .replace('ministra de hacienda en funciones', 'montero cuadrado') \
        .replace('ministro del interior en funciones', 'grande-marlaska gómez') \
        .replace('rodrí guez hernández', 'rodríguez hernández') \
        .replace('ministro de agricultura pesca y alimentación en funciones', 'planas puchades') \
        .replace('ministro de fomento en funciones', 'ábalos meco') \
        .replace('ministra de economía y empresa en funciones', 'calviño santamaría') \
        .replace(' candidato a la presidencia del gobierno', '') \
        .replace('ministro de inclusión  seguridad social y migraciones', 'escrivá belmonte') \
        .replace('ministra de política territorial y función pública', 'darias san sebastián') \
        .replace('ministra de hacienda', 'montero cuadrado') \
        .replace('representante de la asamblea regional de murcia', 'conesa alcaraz') \
        .replace('de olano vela', 'olano vela') \
        .replace('ministro del interior', 'grande-marlaska gómez') \
        .replace('ministro de interior', 'grande-marlaska gómez') \
        .replace('ministro de agricultura pesca y alimentación', 'planas puchades') \
        .replace('ministro de transportes movilidad y agenda urbana', 'ábalos meco') \
        .replace('presidente del gobierno', 'sánchez pérez-castejón') \
        .replace('ministro de sanidad', 'illa roca') \
        .replace('concicao', 'garriga vaz de concicao') \
        .replace('oblanca', 'martínez oblanca') \
        .replace('ministra de igualdad', 'montero gil') \
        .replace('herrer', 'rodríguez herrer') \
        .replace('ministra de industria comercio y turismo', 'maroto illera') \
        .replace('ministro de inclusión seguridad social y migraciones', 'escrivá belmonte') \
        .replace(' moro', 'oramas gonzález-moro') \
        .replace('ministra de asuntos exteriores unión europea y cooperación', 'gonzález laya') \
        .replace('ministro de justicia', 'campo moreno') \
        .replace('vicepresidente segundo del gobierno y ministro de derechos sociales y agenda 2030', 'iglesias turrión') \
        .replace('martínez martínez oblanca', 'martínez oblanca') \
        .replace('sanchez', 'sánchez').replace('castejon', 'castejón') \
        .replace('ministro de inclusión, seguridad social y migraciones', 'escrivá belmonte')

In [8]:
def remove_stopwords(dialog):
    list_specific_stop_words = ['gracias', 'señor', 'señora', 'señorías', 'presidenta', 'ministro', 'ustedes',
                                'usted', 'si', ]
    stop_words = set(stopwords.words('spanish'))

    word_tokens = word_tokenize(dialog)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = [w for w in filtered_sentence if not w in list_specific_stop_words]
        
    return filtered_sentence

In [9]:
def clean_dialogs(dialogs):
    output = []
    for dialog in dialogs:
        speaker = clean_names(clean_mr_mrs(clean_parenthesis(dialog[0])))
        text = remove_stopwords(clean_text(clean_parenthesis(dialog[1])))
        output.append([speaker, text])
    return output

In [10]:
# Almacenamos en una lista, la relacion y la lista de diálogos
congreso = []
for file in os.listdir('files/'):
    if file[-4:] == 'docx':
        doc = docx.Document('files/' + file) 
        dialogs = extract_dialogs(doc)
        sep_dialogs = separate_dialogs(dialogs)
        cl_dialogs = clean_dialogs(sep_dialogs)
        relation = file[:-5]
        congreso.append([relation, cl_dialogs])

#### En este momento tenemos una estructura de este tipo:
['ID_DEBATE', [[DIPUTADO, [PALABRAS]]]]'

In [11]:
def generate_tagger_model(list_words, sw_generate_model):
    jar = 'nlp/stanford-postagger.jar'
    model = 'nlp/spanish.tagger'
    nlp = spacy.load('es_core_news_md')
    pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')
    dialog = list(set(list_words)) 
    max_count  = len(dialog)
    print(max_count)
    
    with tqdm(total=max_count) as pbar:
        if sw_generate_model:
            print('GENERANDO MODELO')
            try:
                os.remove('words.pickle')
                print('MODELO BORRADO')
            except:
                pass
            # taggeamos las palabras de 100 en 100 para no desbordar la memoria
            salto = 100

            current_pos = 0

            while current_pos < max_count :
                try:
                    with open('words.pickle', 'rb') as handle:
                        words = pickle.load(handle)
                        words.extend(pos_tagger.tag(dialog[current_pos:current_pos + salto]))
                        with open('words.pickle', 'wb') as handle:
                            pickle.dump(words, handle, protocol=pickle.HIGHEST_PROTOCOL)
                except Exception as e:
                    print(e)
                    print("CREAMOS NUEVO MODELO")
                    words = pos_tagger.tag(dialog[current_pos:current_pos + salto])

                    with open('words.pickle', 'wb') as handle:
                        pickle.dump(words, handle, protocol=pickle.HIGHEST_PROTOCOL)


                current_pos = current_pos + salto - 1
                pbar.update(current_pos)
            else:
                print("CARGAMOS MODELO")
                with open('words.pickle', 'rb') as handle:
                    words = pickle.load(handle)
                current_pos = max_count
        pbar.update(current_pos)         
    print("MODELO TERMINADO")

    output = []
    for word in words:
        if word[0].isnumeric():
            pass
        else:
            temp = nlp(word[0])[0].lemma_
            if word[1][0:2] in ['nc', 'np', 'aq']:
                if temp not in ['así', 'parte', 'puede', 'hace', 'hacer', 'diputado', 'diputados', '%']:
                    output.append(temp)
            else:
                if word[1][0:1] == 'v':
                    if temp not in ['ser', 'hacer', 'ir', 'decir', 'poder', 'estar', 
                                    'llevar', 'tener', 'querer', 'ser']:
                        output.append(temp)

    return output


SyntaxError: invalid syntax (<ipython-input-11-c10f7c683100>, line 42)

In [ ]:
def generate_list_words(congreso):
    output = []
    for el1 in congreso:
        for el2 in el1[1]:
            output.extend(el2[1])  
    return output

In [ ]:
list_words = generate_list_words(congreso)

In [ ]:
%store -r data
data
print(data)

In [12]:
%%time
tagger_model = generate_tagger_model(list_words, data)

NameError: name 'generate_tagger_model' is not defined

In [ ]:
def load_dialogs (congreso, list_words_tagged):
    print('INCIO CARGA DIALOGOS')
    nlp = spacy.load('es_core_news_md')
    graph = nc.generate_graph()
    matcher = nc.generate_nodeMatcher(graph)
    print('DOCUMENTOS ' + str(len(congreso)))
    for el1 in congreso:
        relation = el1[0]
        print('DIALOGOS ' + str(len(el1[1])))
        for el2 in el1[1]:
            speaker = el2[0].strip()
            diputado = nc.return_diputado(matcher, speaker)
            if diputado is None:
                print("PERSONA NO ENCONTRADA: ")
                print(speaker)
                print(relation)
            else:
                for word in el2[1]:
                    lemma = nlp(word)[0].lemma_
                    if lemma in list_words_tagged:
                        graph.run(nc.insert_palabra(word))
                        graph.run(nc.insert_relation(diputado['apellidos'], word, relation))
                    else:
                        print('PALABRA NO ENCONTRADA')
                        print(lemma)
                        print(relation)

In [ ]:
load_dialogs(congreso, tagger_model)